<a href="https://colab.research.google.com/github/Cyber-Aju/HP/blob/main/HpSolve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.5/187.5 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.8.0-py3-none-any.whl size=258393 sha256=888efe1651d2937a782ab8f49b13084e4a96ab9ffad2b3358ce2cc2eabf7daa9
  Stored in directory: /root/.cache/pip/wheels/6b/b5/da/73f634944e04e625954d101cb175ac1aeb9b29751a37d3383e
Successfully built neo4j


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
import tweepy
import re
import nltk
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import torch
import numpy as np
import pandas as pd
from neo4j import GraphDatabase

In [6]:
# Twitter API keys and tokens
consumer_key = 'WlN1zRbamvD9U0cEqA3i0z3yC'
consumer_secret = 'OKqt67JF3JDE2uSc6eOSUUv6yGVrbCFBgMu3E52b7yiZwKVSQa'
access_token = '1457602236079542272-zBUlvZO6f2qCCvJsfC7QccnYEq2wcX'
access_token_secret = '1XEM9dWZkhnzdBHmkmGuXtWd7Z1Sip2mFxVBvRUv2r6gz'

# Authenticate with Twitter API using Tweepy library
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Define search query for HP products on Twitter
search_query = 'HP PC OR HP Printer OR HP laptop OR HP DESKTOPS OR HP ALL-IN-ONE COMPUTER OR ENVY SERIES'

# Collect tweets containing the search query in English language
tweets = tweepy.Cursor(api.search_tweets,
              q=search_query,
              lang='en',
              count=100).items()
 

In [11]:
# Preprocess text data by removing links, mentions, hashtags, special characters, stop words, and stemming words
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text) # remove links
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#', '', text) # remove hashtags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # remove special characters
    text = text.lower() # convert to lowercase
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words if word not in stop_words]
    text = ' '.join(stemmed_words) # stem words
    return text

# Remove duplicates from tweets and preprocess them
clean_tweets = list(set([preprocess_text(tweet.text) for tweet in tweets]))

Forbidden: ignored

In [15]:
# Load Vader sentiment analysis library
sia = SentimentIntensityAnalyzer()
def get_sentiment(text):
    scores = sia.polarity_scores(text)
    if scores['compound'] > 0.05:
        return 'positive'
    elif scores['compound'] < -0.05:
        return 'negative'
    else:
        return 'neutral'

# Classify tweets as mentioning HP products or not using NaiveBayes classifier
X = clean_tweets
y = np.array([1 if "hp" in tweet.lower() else 0 for tweet in X])
pipeline = Pipeline([
                      ('vectorizer', TfidfVectorizer()),
                      ('clf', MultinomialNB())
                    ])
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)], 
              'clf__alpha': np.linspace(0.5, 1.5, 6)}
grid_search = GridSearchCV(pipeline, parameters, cv=5)
grid_search.fit(X, y)

# Extract the best model from grid search
best_pipeline = grid_search.best_estimator_

# Classify tweets as mentioning HP products or not using the best model
predicted_classes = best_pipeline.predict(clean_tweets)
predicted_probabilities = best_pipeline.predict_proba(clean_tweets)[:, 1]

# Store labeled tweets in a dataframe
labeled_tweets = pd.DataFrame({
    'text': clean_tweets,
    'brand_model': predicted_classes, # 0 for non-HP, 1 for HP
    'brand_model_probability': predicted_probabilities
})

# Load Spacy library for entity extraction
nlp = spacy.load('en_core_web_sm')
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        if ent.label_ in ['PRODUCT', 'ORG']:
            entities.append(ent.text)
    return entities

# Extract product models and features mentioned in tweets using Spacy library
labeled_tweets['entities'] = labeled_tweets['text'].apply(extract_entities)

# Analyze sentiment of tweets using Vader library
labeled_tweets['sentiment'] = labeled_tweets['text'].apply(get_sentiment)

NameError: ignored

In [12]:








# Set up connection to Neo4j instance
uri = 'bolt://localhost:7687'
username = 'Your Username Here'
password = 'Your Password Here'
driver = GraphDatabase.driver(uri, auth=(username, password))

# Define function for creating nodes and edges in the Neo4j knowledge graph
def create_nodes(tx, entities):
    for entity in entities:
        tx.run("MERGE (n:


Entity {name: $name})", name=entity)

def create_edges(tx, source_entity, target_entity, relationship_type):
tx.run("""
MATCH (source:Entity {name: $source_entity})
MATCH (target:Entity {name: $target_entity})
MERGE (source)-[:%s]->(target)
""" % relationship_type, source_entity=source_entity, target_entity=target_entity)
Define function for querying the Neo4j knowledge graph based on a specific issue or topic

def query_knowledge_graph(issue):
with driver.session() as session:
result = session.run("""
MATCH (e1:Entity)-[r]->(e2:Entity)
WHERE e1.name = $issue OR e2.name = $issue
RETURN e1.name AS source, type(r) AS relationship, e2.name AS target
""", issue=issue)
return result
Run queries against the knowledge graph for each entity mentioned in the tweets

for tweet in labeled_tweets.itertuples():
entities = tweet.entities
if tweet.brand_model == 1: # only create nodes and edges for HP products
with driver.session() as session:
session.write_transaction(create_nodes, entities)
for i in range(len(entities)):
for j in range(i+1, len(entities)):
entity1 = entities[i]
entity2 = entities[j]
session.write_transaction(create_edges, entity1, entity2, tweet.sentiment)
Print results of knowledge graph queries for each issue mentioned in the search query

for issue in ['HP PC', 'HP Printer', 'HP laptop']:
print('Results for {}:'.format(issue))
result = query_knowledge_graph(issue)
for record in result:
print('{0} -{1}-> {2}'.format(record['source'], record['relationship'], record['target']))

SyntaxError: ignored